In [ ]:
from dotenv import load_dotenv
import os
token = os.getenv('HUGGINGFACE_TOKEN')

In [1]:
import huggingface_hub
huggingface_hub.login(token=token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /media/joeymeyer/970-evo-plus/.huggingfacecache/token
Login successful


In [2]:
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM, TextGenerationPipeline, AutoConfig, pipeline
import pandas as pd
from peft import PeftModel, PeftConfig
import time
import torch

/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
2024-04-07 10:46:53.095774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 10:46:54.329659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Coul

In [3]:
df = pd.read_csv('golden answers.csv')
prompt_list = df['Golden Question'].to_list()
df.head()

,Index,Golden Answer - SM,Golden Question
0,1,It sounds like a fantastic trip you have plann...,What are your favorite luxury camp meals for a...
1,2,It sounds like a fantastic trip you have plann...,I'm taking a solo trip with my dog around Than...
2,3,Embarking on a cross-Canada road trip with you...,I'm planning a 3-4 week family road trip from ...
3,4,If you're choosing between Seoul and Tokyo for...,What are the main differences between visiting...
4,5,Planning a short solo trip for your birthday i...,Seeking destination and activity suggestions f...


# LLAMA

In [4]:
model_dict = {#'Mistral QLORA Good Data':"soniawmeyer/Mistral_7B_QLORA_SM", #peft requires cuda 11
              'pretrained Llama':"meta-llama/Llama-2-7b-hf", #
            #   'pretrained Mistral':"mistralai/Mistral-7B-Instruct-v0.2", #issue with attention mask??
              'Llama QLORA Chris':'chriztopherton/llama-2-7b-wanderchat', #?
              #'Llama RAFT Berta':'beraht/WanderChat_RAFT',
              'Llama RAFT':'beraht/WanderChat_RAFT_epoch10_step20',
              #'Mistral QLORA Bad Data':"sherrysi/Mistral7B_QLORA_cleaner_r8" #peft requires cuda 11
              }

In [13]:
current_model = 'Llama RAFT'

In [14]:
model_id = model_dict[current_model]
model_name = current_model

In [15]:
start_time = time.time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
end_time = time.time()
duration = end_time - start_time
print(model_name, duration)
with open(f'log/{model_name} load time.txt', 'w') as file:
    file.write(str(duration))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Llama RAFT 369.99686336517334


/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.

In [18]:
answers = []
answer_time = []
response_length = 200

tokenizer.pad_token = tokenizer.eos_token

for prompt in prompt_list:
    start_time = time.time()
    inputs = tokenizer(prompt, return_tensors="pt") #pretrained llama
    input_length = len(inputs.input_ids[0])
    dynamic_length = input_length + response_length
    # Generate
    generate_ids = model.generate(inputs.input_ids, max_length=dynamic_length) #pretrained llama
    answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    end_time = time.time()
    duration = end_time - start_time
    answers.append(answer)
    answer_time.append(duration)
    print(input_length)
    print(prompt)  
    print('\n')
    print(duration)
    print(answer)
    print('\n-----------------------------\n')

51
What are your favorite luxury camp meals for a beginner-friendly 10-mile, 3-day backpacking trip, considering I'm willing to carry 10lbs of food and have basic cookware?


193.19378232955933
What are your favorite luxury camp meals for a beginner-friendly 10-mile, 3-day backpacking trip, considering I'm willing to carry 10lbs of food and have basic cookware?
 I'm looking for a meal that's easy to prepare, doesn't require a lot of fuel, and is delicious.

I'm looking for a meal that's easy to prepare, doesn't require a lot of fuel, and is delicious.

I'm looking for a meal that's easy to prepare, doesn't require a lot of fuel, and is delicious.

I'm looking for a meal that's easy to prepare, doesn't require a lot of fuel, and is delicious.

I'm looking for a meal that's easy to prepare, doesn't require a lot of fuel, and is delicious.

I'm looking for a meal that's easy to prepare, doesn't require a lot of fuel, and is delicious.

I'm looking for a meal that's easy to

--------------

In [19]:
df[f'{model_name} Output'] = answers
df[f'{model_name} Output Time'] = answer_time
df.to_csv(f'\log\{model_name} golden answers output.csv')

In [24]:
df.columns

Index(['Index', 'Golden Answer - SM', 'Golden Question',
       'Llama QLORA Chris Output', 'Llama QLORA Chris Output Time',
       'Llama RAFT Output', 'Llama RAFT Output Time'],
      dtype='object')

In [18]:
del model, tokenizer

# MISTRAL - not working

In [4]:
model_dict = {'Mistral QLORA Good Data':"soniawmeyer/Mistral_7B_QLORA_SM", #peft requires cuda 11
              'pretrained Mistral':"mistralai/Mistral-7B-Instruct-v0.2", #working w code below config set up
              'Mistral QLORA Bad Data':"sherrysi/Mistral7B_QLORA_cleaner_r8" #peft requires cuda 11
              }

In [5]:
current_model = 'Mistral QLORA Good Data'

In [6]:
model_id = model_dict[current_model]
model_name = current_model

In [7]:
adapter_path="soniawmeyer/Mistral_7B_QLORA_SM"
base_model_path=model_dict['pretrained Mistral']

In [9]:
config = AutoConfig.from_pretrained(base_model_path)
config.update({'sliding_window': 4096})
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", config=config)
tokenizer = AutoTokenizer.from_pretrained(model_id, config=config)

pipe = TextGenerationPipeline(model=model, tokenizer=tokenizer)

In [8]:
adapter_path="soniawmeyer/Mistral_7B_QLORA_SM"
base_model_path=model_dict['pretrained Mistral']
config = AutoConfig.from_pretrained(base_model_path)
config.update({'sliding_window': 4096})

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    # low_cpu_mem_usage=True,
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
model = PeftModel.from_pretrained(model, adapter_path)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
    trust_remote_code=True,
    config=config
)
model = model.merge_and_unload()

OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 1 has a total capacity of 7.92 GiB of which 8.19 MiB is free. Including non-PyTorch memory, this process has 7.78 GiB memory in use. Of the allocated memory 7.67 GiB is allocated by PyTorch, and 13.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Assuming `model` is your model instance and it has parameters in BFloat16
for param in model.parameters():
    param.data = param.data.to(torch.float32)  # Convert parameters to float32

In [ ]:
pipe = TextGenerationPipeline(model=model, tokenizer=tokenizer)

In [ ]:
pipe("test me!", max_length=30)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


TypeError: bad operand type for unary -: 'NoneType'

In [14]:
# model_id = model_dict[current_model]
# model_name = current_model
# start_time = time.time()

# adapter_path=model_id
# base_model_path=model_dict['pretrained Mistral']
# config = AutoConfig.from_pretrained(base_model_path)

# model = AutoModelForCausalLM.from_pretrained(
#     base_model_path,
#     torch_dtype=torch.bfloat16,
#     # low_cpu_mem_usage=True,
#     trust_remote_code=True,
# )

# model = PeftModel.from_pretrained(model, adapter_path)

# tokenizer = AutoTokenizer.from_pretrained(
#     base_model_path,
#     trust_remote_code=True,
# )
# model = model.merge_and_unload()

# # Assuming `model` is your model instance and it has parameters in BFloat16
# for param in model.parameters():
#     param.data = param.data.to(torch.float32)  # Convert parameters to float32

# pipe = TextGenerationPipeline(model=model, tokenizer=tokenizer)

# end_time = time.time()
# duration = end_time - start_time
# print(model_name, duration)
# with open(f'{model_name} load time.txt', 'w') as file:
#     file.write(str(duration))



: 

In [28]:
pipe("test me!", max_length=30)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


TypeError: bad operand type for unary -: 'NoneType'

In [ ]:
answers = []
answer_time = []
response_length = 200

for prompt in prompt_list:    
    inputs = tokenizer(prompt, return_tensors="pt") #pretrained llama
    input_length = len(inputs.input_ids[0])
    dynamic_length = input_length + response_length
    # Generate
    start_time = time.time()
    result = pipe(prompt, max_length=dynamic_length)
    answer = result[0]['generated_text']
    end_time = time.time()
    duration = end_time - start_time

    answers.append(answer)
    answer_time.append(duration)

    print(input_length)
    print(prompt)  
    print('\n')
    print(duration)
    print(answer)
    print('\n-----------------------------\n')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


TypeError: bad operand type for unary -: 'NoneType'

In [ ]:
df[f'{model_name} Output'] = answers
df[f'{model_name} Output Time'] = answer_time
df.to_csv(f'{model_name} golden answers output.csv')
df.columns

In [ ]:
del model, tokenizer

# Mistral Pretrained (sliding window config)

In [4]:
model_dict = {#'Mistral QLORA Good Data':"soniawmeyer/Mistral_7B_QLORA_SM", #peft requires cuda 11
              'pretrained Mistral':"mistralai/Mistral-7B-Instruct-v0.2", #working w code below config set up
              #'Mistral QLORA Bad Data':"sherrysi/Mistral7B_QLORA_cleaner_r8" #peft requires cuda 11
              }

In [13]:
# current_model = 'pretrained Mistral'
current_model = 'Mistral QLORA Good Data'

In [26]:
model_id = model_dict[current_model]
model_name = current_model
start_time = time.time()

config = AutoConfig.from_pretrained(model_id)
config.update({'sliding_window': 4096})
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", config=config)
tokenizer = AutoTokenizer.from_pretrained(model_id, config=config)

pipe = TextGenerationPipeline(model=model, tokenizer=tokenizer)

end_time = time.time()
duration = end_time - start_time
print(model_name, duration)
with open(f'log/{model_name} load time.txt', 'w') as file:
    file.write(str(duration))

OSError: soniawmeyer/Mistral_7B_QLORA_SM does not appear to have a file named config.json. Checkout 'https://huggingface.co/soniawmeyer/Mistral_7B_QLORA_SM/main' for available files.

In [27]:
answers = []
answer_time = []
response_length = 200

for prompt in prompt_list:    
    
    dynamic_length = len(prompt)/4 + response_length
    # Generate
    start_time = time.time()
    result = pipe(prompt, max_length=dynamic_length)
    answer = result[0]['generated_text']
    end_time = time.time()
    duration = end_time - start_time

    answers.append(answer)
    answer_time.append(duration)

    print(input_length)
    print(prompt)  
    print('\n')
    print(duration)
    print(answer)
    print('\n-----------------------------\n')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


TypeError: bad operand type for unary -: 'NoneType'

In [17]:
df[f'{model_name} Output'] = answers
df[f'{model_name} Output Time'] = answer_time
df.to_csv(f'\log\{model_name} golden answers output.csv')
df.columns

Index(['Index', 'Golden Answer - SM', 'Golden Question',
       'pretrained Mistral Output', 'pretrained Mistral Output Time'],
      dtype='object')

In [ ]:
del model, tokenizer

# Pipe - Test

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="chriztopherton/llama-2-7b-wanderchat")

2024-04-06 15:21:25.156614: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 15:21:26.396077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
pipe("This restaurant is awesome", max_length=100,  # Increase max_length for longer outputs
                       temperature=0.9,  # Adjust creativity
                       top_k=50,         # Top-k sampling
                       top_p=0.95,       # Top-p (nucleus) sampling, for more diversity
                       num_return_sequences=1)

/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/media/joeymeyer/970-evo-plus/Sonia/WanderChat/venv/lib/python3.11/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'This restaurant is awesome! The food is delicious and the service is top-notch. I'}]

# PEFT - Not Working

In [9]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
config_name = "soniawmeyer/Mistral_7B_QLORA_SM"

config = PeftConfig.from_pretrained(config_name)
model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(model, config_name)
# tokenizer = AutoTokenizer.from_pretrained(config_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
prompt = "What are your favorite luxury camp meals for a beginner-friendly 10-mile, 3-day backpacking trip, considering I'm willing to carry 10lbs of food and have basic cookware?"
inputs = tokenizer(prompt, return_tensors="pt")

In [18]:
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


TypeError: bad operand type for unary -: 'NoneType'